# 2) Entanglement entropy for the one-dimensional Heisenberg model


Consider the one-dimensional Heisenberg model with open boundary conditions:
\begin{equation}
    H = -\sum\limits_{i=1}^{N-1}J_i \vec S_i \cdot \vec S_{i+1}.
\end{equation}
For an even number of sites $N$ and $J_i<0$ (anti-ferromagnetic case), the ground state is non-degenerate.

Now divide the system in two parts $A$ and $B$ with sites $1,\dots, M_A$ and $M_A+1,\dots,N$.


**Entanglement entropy**

For a state
\begin{equation}
    |\psi\rangle = \sum\limits_{i=1}^{\dim_A} \sum\limits_{j=1}^{\dim_B} c_{ij} |{i}\rangle_A |{j}\rangle_B
\end{equation}

the reduced density operator is (with density operator $\hat\rho = |{\psi}\rangle\langle{\psi}|$)
\begin{align}
    \hat\rho_A 
        &= \operatorname{Tr}_B(\underbrace{|{\psi}\rangle\langle{\psi}|}_{=\hat\rho})
        \\
        &= \sum\limits_l \sum\limits_{i,i'} c_{i'l}c_{il}^* |{i'}\rangle_{AA}\langle{i}| 
        \\
        \leadsto \rho_A &= C C^\dagger
\end{align}
where $C$ is the matrix $(c_{ij})_{i=1,\dots,\dim_A; j=1,\dots,\dim_B}$, because $(\rho_A)_{nm} = \sum\limits_l \underbrace{(C)_{nl}}_{=c_{nl}} \underbrace{(C^\dagger)_{lm}}_{=c_{ml}^*}$.

Then the entanglement entropy is
\begin{align}
    S_e 
        &= - \operatorname{Tr}_A [\hat\rho_A \cdot \ln\hat\rho_A] \\
        &= - \sum\limits_\alpha w_\alpha \cdot \ln w_\alpha
\end{align}
with eigenvalues $w_\alpha$ of $\rho_A$.

<blockquote>
Analogous for $\rho_B$ it is:
\begin{align}
    \hat\rho_B 
        &= \operatorname{Tr}_A(\underbrace{|{\psi}\rangle\langle{\psi}|}_{=\hat\rho})
        \\
        &= \sum\limits_l \sum\limits_{j,j'} c_{lj'}c_{lj}^* |{j'}\rangle_{BB}\langle{j}| 
        \\
        (\rho_A)_{nm} &= \sum\limits_l \underbrace{(C)_{ln}}_{=c_{ln}} \underbrace{(C^\dagger)_{ml}}_{=c_{lm}^*}
        \\
        &= \sum\limits_l (C^\dagger)_{ml}(C)_{ln}
        \\
        \leadsto \rho_A &= C^\dagger C
\end{align}
</blockquote>

To numerically figure out $\rho_A$ to calculater $S_e$, we need to find $C$ from a given $|\psi\rangle$.

<blockquote>
The direct method over the definition: Calculate the partial trace of $|\psi\rangle\langle\psi|$ to get $\rho_A$ and then proceed (complicated and includes all steps of the better option below).
</blockquote>

**Calculate $C$ directly from $|\psi\rangle$:** <br/>
Consider $|\psi\rangle$ is given in the computational basis of the joint system as a vector of coefficients $\psi:=\vec\psi$ with
\begin{align}
    |\psi\rangle 
        = \sum\limits_{i=0}^{2^N-1} \psi_i |i\rangle_N
        = \sum\limits_{i=0}^{2^N-1} \psi_i |i_1,i_2,\dots,i_N\rangle
\end{align}

If we now divide the system of $N$ sites in $M_A$ and $M_B = N-M_A$ sites, this can be seen as writing $|i_1,i_2,\dots,i_N\rangle$ as a product state $|i_1,i_2,\dots,i_{M_A}\rangle\otimes |i_{M_A+1},\dots,i_N\rangle$

\begin{align}
    |\psi\rangle 
        &= \sum\limits_{i=0}^{2^N-1} \psi_i \underbrace{|i_1,\dots,i_{M_A}\rangle}_{=:|j\rangle} \otimes \underbrace{|i_{M_A+1},\dots,i_N\rangle}_{=:|l\rangle}
        \\
        &= \sum\limits_{j=0}^{2^{M_A}-1}\sum\limits_{l=0}^{2^{M_B}-1} (C)_{jl} |j\rangle \otimes |l\rangle
\end{align}
with now constructed $C$:
\begin{equation}
    (C)_{jl} = \psi_{dec([j\otimes l]_2)} = \psi_{dec([j_1,\dots,j_{M_A},l_1,\dots,l_{M_B}]_2)} = \psi_{i(j,l)},
\end{equation}
where $dec(x)$ is the decimal representation of binary $x$.


In [168]:
using LinearAlgebra

In [259]:
# some helper functions
n_dual(n, pad=0) = digits(n, base=2, pad=pad)
dec2bin(x, pad=0) = join(digits(x, base=2, pad=pad))
bin2dec(x) = parse(Int, join(reverse(string.(x))), base=2)

bin2dec (generic function with 1 method)

In [287]:
ψ = [1 0 1 0 0 0 0 0] # =̂ |000⟩ + |010⟩ = |0⟩⊗(|00⟩ + |10⟩)

1×8 Matrix{Int64}:
 1  0  1  0  0  0  0  0

In [289]:
M_A = 1 # dim_A = 2^M_A
M_B = 2 # dim_B = 2^M_B

# convert ψ to C:
function ψ2C(ψ, M_A, M_B)
    C = zeros(2^M_A,2^M_B)
    for j in 0:(2^M_A-1)
        for l in 0:(2^M_B-1)
            C[j+1,l+1] = ψ[bin2dec(dec2bin(j,M_A) * dec2bin(l, M_B)) + 1]
            #println("$j, $l: ", dec2bin(j,M_A),",", dec2bin(l, M_B), " → ", C[j+1,l+1])
        end
    end
    return C
end

ψ2C (generic function with 1 method)

In [290]:
C = ψ2C(ψ, 1,2 )

2×4 Matrix{Float64}:
 1.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [194]:
function wₐlog₂wₐ(wₐ)
    if wₐ == 0
        return 0.0 # lim(x→0) x log2(x) = 0
    end
    return wₐ * log2(wₐ)
end

wₐlog₂wₐ (generic function with 1 method)

In [222]:
function Sₑ(ρ)
    return -sum(wₐlog₂wₐ.(eigvals(ρ)))
end

Sₑ (generic function with 1 method)

In [291]:
ρ = 1/2 * C * C' # normalization here for precision

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  0.0

In [292]:
eigvals(ρ)

2-element Vector{Float64}:
 0.0
 1.0

In [286]:
Sₑ(ρ)

1.0